# Jupyter Notebook containing code for the IBM Capstone Project Course

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Code for the Week 3 Peer-review Assignment

## From here on, the code refers to the segmentation and clustering of the neighborhoods of Toronto

### 1st part

In [1]:
import pandas as pd
import numpy as np

In [19]:
# Since the information in the Wikipedia is already in a table (structured), 
# we can use pandas to directly read this information.

temp_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toronto_df = temp_wiki[0]

# Rename the columns to be like the ones shown in the assignment instructions
toronto_df.columns = ['PostalCode','Borough','Neighborhood']

# Drop non-assigned boroughs by first substituting the string by NaN and then droping it
toronto_df.replace('Not assigned', np.NaN, inplace=True)
toronto_df.dropna(inplace=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [22]:
# Check dataframe's shape
toronto_df.shape

(103, 3)

### 2nd part